In [1]:
# from Policy1_demo_workloads import wrk_different_requests
# from iDynamicsPackagesModules.SchedulingPolicyExtender.my_cluster_utils import (
#     gather_all_nodes, build_nodeinfo_objects,
#     build_podinfo_objects, gather_all_pods)

# from iDynamicsPackagesModules.GraphDynamicsAnalyzer.graph_builder import build_call_graph
from iDynamicsPackagesModules.Evaluations.Policy1_eval_Graph_dynamics.Policy1_Call_Graph_Aware import Policy1CallGraphAware
# from iDynamicsPackagesModules.Evaluations.Policy2_eval_Delay_dynamics.Policy2_Latency_Aware import Policy2LatencyAware
# from iDynamicsPackagesModules.Evaluations.Policy3_eval_Bandwidth_dynamics.Policy3_Bandwidth_Aware import Policy3BandwidthAware

# from Policy1_demo_workloads import run_workload_varing_callGraph


Pods already exist in the namespace. Skipping DaemonSet deployment.


In [2]:

import time
import threading


def evaluate_policy1_scheudlder():
    # Initialize the scheduler with necessary parameters
    app1_prom_url = "http://10.105.116.175:9090"
    app1_qos_target = 150  # QoS target in milliseconds
    # time_window is the look_back window for the average response time
    app1_time_window = 1  # Time window in minutes,
    app1_namespace = 'social-network'
    app1_response_code = '200'  # HTTP response code to consider

    # Create an instance of the scheduler
    policy1 = Policy1CallGraphAware()
    policy1.initialize_policy(dynamics_config={"traffic_pairs": {}}, prom_url=app1_prom_url, 
                            qos_target = app1_qos_target, time_window = app1_time_window, 
                            namespace = app1_namespace, response_code = app1_response_code)

    # Run the scheduler
    while True:
        policy1.run()
        time.sleep(20) # Run the scheduler to regularly  check if the QoS if violated every 30 seconds

time.sleep(60*16*2 + 60*3)
evaluate_policy1_scheudlder()

# (1) before running the Policy1 scheduler, we run the workload for the first time 
# to obtain performance in default k8s scheduler

# run_workload_varing_callGraph(each_wrk2_duration='3m') # eg., "30s", "1m", 5m", "10m", "15m"
#sleep for 2 mins
# time.sleep(180)

# (2) Conccurently run evaluate_policy1_scheudlder() and run_workload_varing_callGraph()
# workload_thread = threading.Thread(target = run_workload_varing_callGraph, kwargs={'each_wrk2_duration':'3m'})
# scheduler_thread = threading.Thread(target=evaluate_policy1_scheudlder)
# workload_thread.start()
# scheduler_thread.start()

# # Wait for both threads to finish
# workload_thread.join()
# scheduler_thread.join()



Average response time = 12.21 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.23 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.83 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.65 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.55 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.83 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 13.28 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 13.31 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 12.90 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 13.78 ms
Trigger = False
[Policy1CallGraphAware] No migration triggered.
Average response time = 14.32 

KeyboardInterrupt: 

In [ ]:
app1_prom_url = "http://10.105.116.175:9090"
app1_qos_target = 150  # QoS target in milliseconds
# time_window is the look_back window for the average response time
app1_time_window = 1  # Time window in minutes,
app1_namespace = 'social-network'
app1_response_code = '200'  # HTTP response code to consider

# Create an instance of the scheduler
policy1 = Policy1CallGraphAware()
policy1.initialize_policy(dynamics_config={"traffic_pairs": {}}, prom_url=app1_prom_url, 
                        qos_target = app1_qos_target, time_window = app1_time_window, 
                        namespace = app1_namespace, response_code = app1_response_code)

# Run the scheduler
while True:
    policy1.run()
    time.sleep(20) # Run the scheduler to regularly  check if the QoS if violated every 30 seconds

In [ ]:
# 1) Gather cluster data
raw_nodes = gather_all_nodes()
candidate_nodes = build_nodeinfo_objects(raw_nodes)

# Gather pod data for each policy
# Deploye three instances of social-network application in different namespaces
# each policy will be applied to each namespace
raw_pods_Policy1 = gather_all_pods(namespace="social-network")
pods_Policy1 = build_podinfo_objects(raw_pods_Policy1)

raw_pods_Policy2 = gather_all_pods(namespace="social-network2")
pods_Policy2 = build_podinfo_objects(raw_pods_Policy2)

raw_pods_Policy3 = gather_all_pods(namespace="social-network3")
pods_Policy3 = build_podinfo_objects(raw_pods_Policy3)

# 2) (Optional) Build a call graph from 'graph_builder.py'
#    This might be used by Policy1 to place heavily communicating microservices.
G = build_call_graph(namespace="social-network")
# Convert the NetworkX graph to a simple dict: (src, dst) -> traffic_kb
traffic_matrix = {}
for (u, v, data) in G.edges(data=True):
    traffic_matrix[(u, v)] = data.get("weight", 0.0)  # store as KB

# 3) Initialize each policy with scenario-specific config
policy1 = Policy1CallGraphAware()
policy1.initialize_policy(dynamics_config={"traffic_matrix": traffic_matrix})

policy2 = Policy2LatencyAware()
policy2.initialize_policy(dynamics_config={"latency_threshold": 5.0})  # example

policy3 = Policy3BandwidthAware()
policy3.initialize_policy(dynamics_config={"high_traffic_threshold": 300.0})  # example

# 4) Choose which scenario you want to test
# Scenario 1: call-graph dynamics => we expect policy1 to shine
print("=== Scenario1: Call-Graph_Aware ===")
decisions_s1 = policy1.schedule_all(pods_Policy1, candidate_nodes)
for dec in decisions_s1:
    print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

# Scenario 2: cross-node delays => policy2 might do better
print("\n=== Scenario2: Latency_Aware ===")
decisions_s2 = policy2.schedule_all(pods_Policy2, candidate_nodes)
for dec in decisions_s2:
    print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")

# Scenario 3: cross-node bandwidth changes => policy3 might do better
print("\n=== Scenario3: Bandwidth_Aware ===")
decisions_s3 = policy3.schedule_all(pods_Policy3, candidate_nodes)
for dec in decisions_s3:
    print(f"  Pod {dec.pod_name} -> Node {dec.selected_node}")
